In [ ]:
#拟合效果不好 可能是网络深度不够 或者没有优化？
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

train_image_path = glob.glob('./data_set/dc_2000/train/*/*.jpg')
# glob来提取 用*代表任意

train_image_label = [int(p.split('\\')[1] == 'cat') for p in train_image_path]
# 用列表推导式 int之后的true为1

# print(train_image_label[:5],train_image_label[-5:])

def load_preprosess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3) # 解码
    image = tf.image.resize(image,[360,360])# 不规则的图像进行缩放
    image = tf.image.random_crop(image,[256,256,3])#随机裁剪 写出通道数
    image = tf.image.random_flip_left_right(image)#左右翻转
    image = tf.image.random_flip_up_down(image)#上下翻转
    #brightness contrast 也可以用来加强数据
    image = tf.cast(image,tf.float32)# 转化数据类型
    image = image/255# 归一化
    label = tf.reshape(label,[1])
    return image,label
# tf.image.convert_image_dtype 如果原数据不是float转为float并做归一化

#test上不做图片增强
def load_preprosess_test_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3) # 解码
    image = tf.image.resize(image,[256,256])# 不规则的图像进行缩放
    image = tf.cast(image,tf.float32)# 转化数据类型
    image = image/255# 归一化
    label = tf.reshape(label,[1])
    return image,label


train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,train_image_label))

AUTOTUNE = tf.data.experimental.AUTOTUNE
#AUTUTUNE根据你的计算机自动使用并行运算
#AUTOTUNE让计算机自己决定

train_image_ds = train_image_ds.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)
#num_parallel_calls 是否用并行运算并选择个数
#map把函数映射在集合上面
#print(train_image_ds)

BATCH_SIZE = 32
train_count = len(train_image_path)
train_image_ds = train_image_ds.shuffle(train_count).batch(BATCH_SIZE)
train_image_ds = train_image_ds.prefetch(AUTOTUNE)
#加快运行速度 提前读取下一批数据
imgs , labels = next(iter(train_image_ds))
#生成器迭代


test_image_path = glob.glob('./data_set/dc_2000/test/*/*.jpg')
#构建测试集
test_image_label = [int(p.split('\\')[1] == 'cat') for p in test_image_path]
test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path,test_image_label))
test_image_ds = test_image_ds.map(load_preprosess_test_image() , num_parallel_calls=AUTOTUNE)
test_image_ds = test_image_ds.batch(BATCH_SIZE)
test_image_ds = test_image_ds.prefetch(AUTOTUNE)
#提速

#可以增加网络的深度和卷积的参数来提高拟合效果 增加宽度更容易出现过拟合效果 在Sequential模型下面每一层的卷积核不能太少
#添加tf.keras.layers.BatchNormalization()正则化 防止过拟合 放在Conv2D后面

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),input_shape=(256,256,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.GlobalAveragePooling2D(),
    #减少参数
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(1)
]
)
#没有训练

pred = model(imgs)

predict = np.array([p[0].numpy() for p in  tf.cast(pred>0,tf.int32)])
real = np.array([l[0].numpy for l in labels])

#自定义训练
loss = tf.keras.losses.BinaryCrossentropy()
#loss为可调用的参数
optimizer = tf.keras.optimizers.Adam()

epoch_loss_avg = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.Accuracy()


epoch_loss_avg_test = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.Accuracy()

def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels,pred)
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    epoch_loss_avg(loss_step)
    train_accuracy(labels,tf.cast(pred > 0,tf.int32))

def test_step(model ,images,labels):
    pred = model.predict(images)
    loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels,pred)
    epoch_loss_avg_test(loss_step)
    test_accuracy(labels,tf.cast(pred > 0, tf.int32))


train_loss_results = []
train_acc_results = []

test_loss_results = []
test_acc_results = []

num_epochs = 30

for each in range(num_epochs):
    for imgs1,labels1 in train_image_ds:
        train_step(model,imgs1,labels1)
        print('.',end='')#显示一次训练完成
        train_loss_results.append(epoch_loss_avg.result())
        train_acc_results.append(train_accuracy.result())
    for imgs_ , labels_ in test_image_ds:
        test_step(model,imgs_,labels_)
        test_loss_results.append(epoch_loss_avg_test.result())
        test_acc_results.append(test_accuracy.result())
    print()
    print('Epoch:{}:loss:{:.3f}, accuracy:{:.3f}, test_loss:{:.3f}, test_accuracy:{:.3f}'.format(
        each+1,
        epoch_loss_avg.result(),
        train_accuracy.result(),
        epoch_loss_avg_test.result(),
        test_accuracy.result()
    ))#打印结果
    epoch_loss_avg.reset_states()
    train_accuracy.reset_states()
    epoch_loss_avg_test.reset_states()
    test_accuracy.reset_states()
    #重置列表

